In [4]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

--2023-10-21 16:07:00--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’

nsight-systems-2023 100%[===================>] 302.99M   207MB/s    in 1.5s    

2023-10-21 16:07:01 (207 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’ saved [317705436/317705436]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 https://ppa.launchpadco

<h1><div align="center">Zarządzanie pamięcią przyspieszonych aplikacji z CUDA C/C++ Unified Memory</div></h1>

W [*CUDA Best Practices Guide*](http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations), przedstawiony jest cykl projektowy zwany **APOD**: **A**ssess, **P**arallelize, **O**ptimize, **D**eploy (Ocena, Paralelizacja, Optymalizacja, Deploy) z którym warto się zapoznać. Krótko mówiąc, APOD zaleca iteracyjny proces projektowania, w którym programiści mogą stopniowo ulepszać swoje aplikacje. Nabierając doświadczenia, można stosować bardziej zaawansowane techniki optymalizacji.

Na tych zajęciach przedstawione zostanie narzędzie wierszaq poleceń Nsight Systems **nsys** do pomiaru jakości i wydajności aplikacji, a także identyfikowania możliwości dalszej optymalizacji. Pozwoli to na stopniowe stosowanie ulepszeń na podstawie poznanych technik. Wiele technik będzie dotyczyło **CUDA Unified Memory**. Zrozumienie zarządzania tą pamięcią jest podstawową umiejętnością programistów CUDA i stanowi początek dla wielu bardziej zaawansowanych technik zarządzania pamięcią.

---
## Wymagania

Aby sprawnie zrealizować dzisiejsze zadanie, powinniście już umieć:
- Pisać, kompilować i uruchamiać programy w języku C/C++, które zarówno wywołują funkcje na CPU oraz **uruchamiają kernele** na GPU.
- Kontrolować równoległą **hierarchię wątków** wykorzystując **konfigurację wykonania**.
- Przerabiać pętle, aby iteracje wykonywały się równolegle.
- Alokować i zwalniać pamięć dostepną zarówno dla procesorów jak i kart graficznych.

---
## Cele

Po dzisiejszych zajęciach powinniście być w stanie:

- Wykorzystywać Nsight Systems do profilowania wydajności przyspieszonych aplikacji.
- Lepiej zrozumieć **streaming multiprocessors**, aby optymalizować konfiguracje wykonawcze.
- Zruzumieć zachowanie **unified memory** w odniesieniu do różnego rodzaju błędów i migracji danych.
- Wykorzystywać **asynchronous memory prefetching** do zmniejszania liczby błędów i problemów z migracją danych.
- Stosować cykl projektowy APOD do przyspieszania i wdrażania przyspieszonych aplikacji.

---
## Optymalizacje z wykorzystaniem programu NVIDIA Command Line Profiler

Jednym ze sposobów na upewnienie się, że próby optymalizacji kodu są skuteczne jest profilowanie aplikacji pod kątem informacji o ich wydajności. `nsys` to narzędzie wiersza poleceń Nsight Systems. Jest dostarczany wraz z innymi narzędziami CUDA i umożliwia właśnie profilowanie przyspieszonych aplikacji.

`nsys` jest łatwy w uzyciu. Jego najbardziej podstawowym zastosowaniem jest po prostu przekazanie mu ścieżki do pliku wykonywalnego, skompilowanego z użyciem `nvcc`. `nsys` wykona kod aplikacji, a następnie wydrukuje podsumowanie jego działania wykorzystujące GPU, wywołania API CUDA, a także informacje na temat **unified memory**.

Podczas przyspieszania aplikacji czy też optymalizacji istniejących już rozwiązań należy stosować różne techniki. Przede wszystkim należy profilować aplikacje po dokonaniu zmian i odnotowywać ich wpływ na wydajność. Na podstawowym etapie należy dość często dokonywać profilowania, aby zapoznać się z narzędziami oraz nauczyć się, w jaki sposób określone zmiany w kodzie CUDA wpływają na jego rzeczywistą wydajność. Jeśli dokonamy wielu zmian, a nastepnie profilowania, ciężko będzie dojść do tego co miało większy wpływ na przyspieszenie aplikacji.

### Ćwiczenie: Profilowanie aplikacji za pomocą nsys
01-vector-add.cu jest podstawową wersją wykorzystującą GPU w celu przyspieszenia programu dodającego wektory. Pierwsza komórka wykonania kodu skompiluje (i uruchomi) program dodawania wektorów. Druga komórka wykonania kodu będzie profilować plik wykonywalny, który właśnie został skompilowany, przy użyciu `nsys profile`.

`nsys profile` wygeneruje plik raportu `qdrep`, który można użyć na różne sposoby. Używamy tutaj flagi `--stats=true`, aby wskazać, że chcemy wyświetlić podsumowanie statystyk takie jak:

- Szczegóły konfiguracji
- Szczegóły generowania plików
- **Statystyki API CUDA**
- **Statystyki kerneli CUDA**
- **Statystyki operacji pamięci CUDA (czas i rozmiar)**
- Statystyki interfejsu API środowiska wykonawczego systemu operacyjnego

Po profilowaniu aplikacji odpowiedz na następujące pytania, korzystając z informacji wyświetlanych w sekcji `CUDA Kernel Statistics` wyniku profilowania:

- Jak nazywał się jedyny kernel CUDA wywoływany w tej aplikacji?
- Ile razy się uruchomił?
- Jak długo trwało jego uruchomienie? Należy zapisać gdzieś ten czas, aby śledzić wpływ dokonywanych później optymalizacji na jego przyspieszenie.

In [1]:
!nvcc -o single-thread-vector-add 01-vector-add.cu -run

Success! All values calculated correctly.


In [5]:
!nsys profile --stats=true ./single-thread-vector-add

Success! All values calculated correctly.
Generating '/tmp/nsys-report-7ef3.qdstrm'
[1/8] [========================100%] report1.nsys-rep
[2/8] [========================100%] report1.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)        Name     
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  --------------
     98.1    4,714,571,831        246  19,164,926.1  10,078,069.5     5,316  103,206,338  27,469,776.0  poll          
      1.2       58,263,697        531     109,724.5      13,657.0       367   18,306,522     882,019.1  ioctl         
      0.6       27,143,918         24   1,130,996.6       6,360.0       838   10,255,830   3,049,287.0  mmap          
      0.0        1,365,856         31      44,059.9 

Warto wspomnieć, że domyślnie `nsys profile` nie nadpisuje istniejącego pliku raportu. Ma to na celu zapobieganie przypadkowej utracie pracy podczas profilowania. Jeśli z jakiegokolwiek powodu wolisz nadpisać istniejący plik raportu, na przykład podczas szybkich zmian, możesz podać flagę `-f` do `profilu nsys`, aby umożliwić nadpisanie istniejącego pliku raportu.

### Ćwiczenie: Optimize and Profile

Na początek należy zmienić konfigurację wykonania 01-vector-add.cu, aby wykorzystywała więcej bloków i wątków. Nastepnie zrekompilować i zprofilować zmiany z wykorzystaniem `nsys profile --stats=true`. Należy zwrócić uwagę czy tak podstawowa zmiana miała wpływ na czas wykonywania.

In [ ]:
!nvcc -o multi-thread-vector-add 01-vector-add.cu -run

Success! All values calculated correctly.


In [ ]:
!nsys profile --stats=true ./multi-thread-vector-add

Success! All values calculated correctly.

The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-043f-17fc-feb1-4bd9.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-043f-17fc-feb1-4bd9.qdrep"
Exporting 18508 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-043f-17fc-feb1-4bd9.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls     Average       Minimum      Maximum            Name         
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    55.3      255,165,347          3   85,055,115.7       23,926  255,080,661  cudaMallocManaged    
    39.6      182,673,357          1  182,673,357.0  182,673,357  182,673,357  cudaDeviceSynchronize
     5.1       2

### Ćwiczenie: Optimize Iteratively

W tym ćwiczeniu należy dokonać kilku edycji konfiguracji wykonawczej 01-vector-add.cu, a następnie profilowania aby zobaczyć ich wpływ na wydajność. Kroki do zrealizowania:

- Przemyśl od 3 do 5 różnych sposobów zmiany konfiguracji wykonawczej, pamiętając o różnych kombinacjach rozmiarów bloków i grid.
- Dokonaj edycji 01-vector-add.cu na jeden z wybranych sposobów.
- Zkompiluj i dokonaj profilowania.
- Odnotuj czas wykonywania po zmianach.
- Powtórz edycję -> profilowanie -> odnotowanie wyników dla każdego wymyślonego sposobu.

Która konfiguracja wykonawcza przyniosła najlepsze efekty?

In [ ]:
!nvcc -o iteratively-optimized-vector-add 01-vector-add.cu -run

In [ ]:
!nsys profile --stats=true ./iteratively-optimized-vector-add

---
## Streaming Multiprocessors and Querying the Device

Dostępny sprzęt GPU ma ogromne znaczenie na możliwości optymalizacji i przyspieszania aplikacji. Ważnymi elementami są **streaming multiprocessors**. Prezentacja `p1.pptx` przedstawia nadchodzący materiał na wysokim poziomie.


### Streaming Multiprocessors and Warps

Procesory GPU, na których działają aplikacje CUDA mają jednostki przetwarzania zwane **streaming multiprocessors** (SM). Podczas uruchamiania kerneli, bloki wątków są przekazywane do SM w celu wykonania. Jednym z podstawowych sposobów na zwiększenie liczby operacji równoległych, które mają wpływ na wzrost wydajności, jest *dobieranie liczby bloków będących wielokrotnością liczby SM na danym GPU.*

Ponadto SM tworzą, zarządzają, planują i wykonują grupy wątków z danego bloku zwane **warps**. **Warps** składają się z 32 wątków. [Jest to temat zaawansowany i warty zapoznania](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation). Warto jednak wspomnieć, że wzrost wydajności można też osiągnąć poprzez *ustalanie wielkości bloku tak, aby liczba jego wątków była wielokrotnością 32.*

### Wysyłanie zapytań o właściwości urzadzenia GPU

Liczba SM różni się na GPU w zależności od danej architektury. W związku z tym nie powinna znajdować się w kodzie jako zmienna statyczna.

Ponizszy kod pokazuje, jak w CUDA C/C++ uzyskać strukturę, która zawiera wiele właściwości dotyczących aktualnie aktywnego GPU, w tym liczbę SM:

```cpp
int deviceId;
cudaGetDevice(&deviceId);                  // `deviceId` now points to the id of the currently active GPU.

cudaDeviceProp props;
cudaGetDeviceProperties(&props, deviceId); // `props` now has many useful properties about
                                           // the active GPU device.
```

### Ćwiczenie: Właściwości GPU

Obecnie 01-get-device-properties.cu zawiera wiele nieprzypisanych zmiennych i wyświetla nic nieznaczące informacje, które powinny wyświetlać szczegółowe informacje o aktualnie aktywnym GPU.

Dokonaj refaktoryzacji, aby 01-get-device-properties.cu wyświetlały szczegółowe informacje na temat wykorzystywanego urządzenia. Więcej na ten temat można znaleźć w [CUDA Runtime Docs](http://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html). Najważniejsze informacje to:
- Możliwości obliczeniowe (Compute Capability).
- Liczba Streaming Multiprocessors.
- Rozmiar Warp (Warp Size).

In [ ]:
!nvidia-smi

Wed Nov 16 11:51:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc -o get-device-properties 01-get-device-properties.cu -run

Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32


### Ćwiczenie: Optymalizacja 01-vector-add.cu

Wykorzystaj zdobytą wiedzę, aby sprawdzić liczbę SM na wykorzystywanym urządzeniu, a następnie dokonaj refaktoryzacji kernela `addVectorsIno`, aby uruchamiał się z grid zawierającym liczbę bloków będącą wielokrotnością liczby SM na urządzeniu.

W zależności od wcześniej dokonanych zmian, ta refaktoryzacja powinna znacząco poprawić (ale nie musi) wydajność kernela. Pamiętaj o użyciu `nsys profile` i ocenić zmiany wydajności.

In [ ]:
!nvcc -o sm-optimized-vector-add 01-vector-add.cu -run

Success! All values calculated correctly.


In [ ]:
!nsys profile --stats=true ./sm-optimized-vector-add

Success! All values calculated correctly.

The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-0ae5-3f79-c533-a433.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-0ae5-3f79-c533-a433.qdrep"
Exporting 19786 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-0ae5-3f79-c533-a433.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls     Average       Minimum      Maximum            Name         
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    65.6      263,292,176          3   87,764,058.7       16,173  263,221,698  cudaMallocManaged    
    28.5      114,466,845          1  114,466,845.0  114,466,845  114,466,845  cudaDeviceSynchronize
     5.9       2

---
## Unified Memory

Do tej pory pamięć przeznaczona do użytku przez kod hosta lub urządzenia była przydzielana za pomocą `cudaMallocManaged`, czyli migracja pamięci była automatyczna. Ułatwia to programowanie, ponieważ nie trzeba zagłębiać się w szczegóły jak **Unified Memory** (**UM**) prydziela rzeczywiste zadania `cudaMallocManaged`.

`nsys profile` dostarcza szczegółowych informacji na temat zarządzania UM w akcelerowanych aplikacjach, a ich wykorzystanie w połączeniu z bardziej szczegółowym zrozumieniem działania UM zapewnia dodatkowe możliwości optymalizacji aplikacji.

Prezentacja `p2.pptx` przedstawia tę ideę na wysokim poziomie.

### Migracja Unified Memory

Po przydzieleniu UM, pamięć nie znajduje się jeszcze ani na hoście ani na urządzeniu. Gdy host lub urządzenie spróbuje uzyskać dostęp do pamięci, wystapi błąd zwany [page fault](https://en.wikipedia.org/wiki/Page_fault), a host lub urządzenie będzie przeprowadzało partiami migrację potrzebnych danych. Dzieje się tak za każdym razem, kiedy aplikacja będzie odwoływała się do pamięci GPU lub CPU, ale nie znajduje się ona jeszcze na żądanym urzadzeniu.

Możliwość stronicowania błędów (`page fault`) i migracji pamięci na żądanie jest niezwykle pomocna podczas programowania przyspieszonych aplikacji. Ponadto w przypadku kiedy niewiadomo nad którymi danymi trzeba będzie pracować dopóki aplikacja nie zostanie uruchomiona lub aplikacja będzie wykorzystywała wiele urządzeń GPU, migracja pamięci na rządanie jest niezwykle korzystna.

Przykładowo, gdy potrzeby w zagresie danych są znane przed uruchomieniem i wymagane są duże, ciągłe bloki pamięci, `page fault` i migracja danych na rządanie wiąże się z konsekwencjami, których lepiej byłoby uniknąć.

Dalsza częśc skupia się na zrozumieniu migracji na żądanie i sposobom jej identyfikowania w danych wyjściowych profilera.

### Ćwiczenie: Migracja UM i Page Faulting

`nsys profile` dostarcza informacje na temat zachowania UM dla profilowanej aplikacji. W tym ćwiczeniu należy wprowadzić kilka modyfikacji i użyć `nsys profile` po każdej zmianie, aby zbadać jak zachowuje się migracja danych UM.

`01-page-faults.cu` zawiera `hostFunction` i `gpuKernel`, które moga być użyte do inicjalizacji elementów `2<<24` wektora liczbą `1`. Obecnie nie jest używana ani funkcja hosta, ani kernel GPU.

Odpowiedz na poniższe pytania, biorąc pod uwagę wiedzę na temat UM. Należy zacząć od postawienia hipotezy jaki rodzaj błędu `page fault` powinien się wydarzyć, a następnie dokonać edycji jednej lub obu z dwóch dostarczonych funkcji w `01-page-faults.cu` aby przetestować tę hipotezę.

Aby tego dokonać należy skompilować i sprofilować kod i zapisać wyniki uzyskane z `nsys profile --stats=true`. Należy zwrócić uwagę na:

- Czy w danych wyjściowych znajduje się sekcja _CUDA Memory Operation Statistics_?
- Jeśli tak, czy wskazuje migracje z hosta na urządzenie (HtoD) lub z urządzenia na host (DtoH)?
- W przypadku migracji, co dane wyjściowe mówią o liczbie występujących _Operations_? Jeśli występuje wiele małych operacji migracji, może być to znak że wystepują błędy `page faulting` z małymi migracjami pamięci występującymi za każdym razem, gdy w żądanej lokalizacji występuje błąd stronicowania.

Pytania na które należy odpowiedzieć (w razie problemów rozwiązania można znaleźć w folderze _Solutions_ dla odpowiednich pytań):

- Czy istnieją dowody na migrację pamięci i/lub `page faulting` gdy tylko CPU uzyskuje dostęp do UM?
- Czy istnieją dowody na migrację pamięci i/lub `page faulting` gdy tylko GPU uzyskuje dostęp do UM?
- Czy istnieją dowody na migrację pamięci i/lub `page faulting`, gdy UM jest najpierw uzyskiwana przez procesor, a następnie przez GPU?
- Czy istnieją dowody na migrację pamięci i/lub `page faulting`, gdy UM jest uzyskiwana najpierw przez GPU, a następnie przez CPU?

In [ ]:
!nvcc -o page-faults 01-page-faults.cu -run

In [ ]:
!nsys profile --stats=true ./page-faults


The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-b99c-e03e-94bb-485c.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-b99c-e03e-94bb-485c.qdrep"
Exporting 12152 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-b99c-e03e-94bb-485c.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls     Average       Minimum      Maximum            Name         
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    79.3      245,410,749          1  245,410,749.0  245,410,749  245,410,749  cudaMallocManaged    
    18.4       56,973,233          1   56,973,233.0   56,973,233   56,973,233  cudaDeviceSynchronize
     2.3        7,204,996          1    7,204,996.0    7,2

### Ćwiczenie: Zachowanie UM w 01-vector-add.cu

Analizując kod `01-vector-add.cu` należy zastanowić się jakiego rodzaju migracji pamięci lub błędy stronicowania moga wystąpić. Informacje te można obserwując sekcję _CUDA Memory Operation Statistics_ wyświetloną przez `nsys profile`. Czy można wyjaśnić rodzaje migracji i liczbę ich operacje na podstawie kodu?

In [ ]:
!nsys profile --stats=true ./sm-optimized-vector-add

### Ćwiczenie: Inicjalizacja wektora w kernelu

Kiedy `nsys profile` podaje ilość czasu potrzebnego do wykonania kernela, błędy stronicowania między hostem a urządzeniem oraz migracje danych, które występują podczas wykonywania tego kernela są uwzględniane w wyświetlanym czasie wykonania.

Mając to na uwadze, zrefaktoryzuj funkcję hosta `initWith` w 01-vector-add.cu tak, aby zamiast tego była kernelem CUDA, inicjując przydzielony wektor równolegle na GPU. Po pomyślnym skompilowaniu i uruchomieniu refaktoryzowanej aplikacji, ale przed jej profilowaniem, rozważ następujące kwestie:

- Jak dokonane zmiany mogą wpłynąć na zachowanie migracji UM?
- Jak dokonane zmiany mogą wpłynąć na czas wykonywania `addVectorsInto`?

In [ ]:
!nvcc -o initialize-in-kernel 01-vector-add.cu -run

Device ID: 0	Number of SMs: 40
Success! All values calculated correctly.


In [ ]:
!nsys profile --stats=true ./initialize-in-kernel

Device ID: 0	Number of SMs: 40
Success! All values calculated correctly.

The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-cd3c-96eb-a5e5-4946.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-cd3c-96eb-a5e5-4946.qdrep"
Exporting 7313 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-cd3c-96eb-a5e5-4946.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum            Name         
 -------  ---------------  ---------  ------------  ---------  -----------  ---------------------
    77.7      255,297,074          3  85,099,024.7     22,697  255,210,353  cudaMallocManaged    
    15.7       51,770,695          2  25,885,347.5  1,708,237   50,062,458  cudaDeviceSynchroni

---
## Asynchronous Memory Prefetching

Zaawansowana technika zmniejszająca narzut związany z błędami stronicowania i migracjami pamięci na żądanie, zarówno w transferach pamięci między hostami jak i innymi urzadzeniami, nazywana jest **asynchronous memory prefetching** (_asynchroniczne pobieranie z wyprzedzeniem_). Korzystanie z tej techniki umożliwia programistom asynchroniczną migrację UM do dowolnego procesora lub urządzenia GPU w systemie w tle, przed użyciem jej przez kod aplikacji. W ten sposób można zwiększyć wydajność kerneli GPU i funkcji procesora, dzięki ograniczeniu `page faults` i narzutu związanego z migracją danych na żądanie.

**Asynchronous memory prefetching** często dokonuje migracji danych w większych porcjach, a zatem z mniejsza ilością migracji niż w przypadku migracji na żadanie. Sprawia to, że doskonale pasuje to, gdy potrzeby w zakresie dostępu do danych są znane przed uruchomieniem.

CUDA ułatwią tę technikę w łatwy sposó z wykorzystaniem funkcji `cudaMemPrefetchAsync`. Poniżej znajduje się przykład użycia:

```cpp
int deviceId;
cudaGetDevice(&deviceId);                                         // The ID of the currently active GPU device.

cudaMemPrefetchAsync(pointerToSomeUMData, size, deviceId);        // Prefetch to GPU device.
cudaMemPrefetchAsync(pointerToSomeUMData, size, cudaCpuDeviceId); // Prefetch to host. `cudaCpuDeviceId` is a
                                                                  // built-in CUDA variable.
```

### Ćwiczenie: Prefetch Memory

Aktualnie program `01-vector-add.cu` powinien nie tylko uruchamiać kernel CUDA w celu dodania 2 wektorów do trzeciego, z których wszystkie sa przydzielane za pomocą `cudaMallocManaged`, ale powinien również inicjalizować każdy z 3 wektorów równolegle w kernelu CUDA. Jeśli z jakiegoś powodu aplikacja nie wykonuje powyższych czynności należy zapoznać się z `01-vector-add-init-in- kernel-solution.cu` aby odzwierciedlić jego funkcjonalność we własnym programie.


Wykorzystując `cudaMemPrefetchAsync` dokonaj poniższych zmian w `01-vector-add.cu`, aby sprawdzić jaki może mieć wpływ na migrację pamięci i `page faults`:

- Co się stanie, gdy pobierzesz z wyprzedzeniem jeden z zainicjowanych wektorów do urządzenia?
- Co się stanie, gdy pobierzesz z wyprzedzeniem dwa zainicjowane wektory do urządzenia?
- Co się stanie, gdy pobierzesz z wyprzedzeniem wszystkie trzy zainicjowane wektory do urządzenia?

Przed kompilacją zastanów się jak zachowa się UM (zwłaszcza page faulting), a także wpływ na czas uruchomienia inicjalizacji kernela.

In [ ]:
!nvcc -o prefetch-to-gpu 01-vector-add.cu -run

Success! All values calculated correctly.


In [ ]:
!nsys profile --stats=true ./prefetch-to-gpu

Success! All values calculated correctly.

The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-24c5-5447-4a22-b068.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-24c5-5447-4a22-b068.qdrep"
Exporting 6827 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-24c5-5447-4a22-b068.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum            Name         
 -------  ---------------  ---------  ------------  ---------  -----------  ---------------------
    91.3      260,053,786          3  86,684,595.3     13,812  259,999,540  cudaMallocManaged    
     6.4       18,088,459          3   6,029,486.3    834,033   16,326,087  cudaFree             
     1.7        4,855,971   

### Ćwiczenie: Prefetch Memory spowrotem do CPU

Ponownie należy wykorzystać asynchroniczne pobieranie z wyprzedzeniem, tym razem spowrotem do CPU, dla funkcji która sprawdza poprawność kernela `addVectorInto`. Po refaktoryzacji zapoznaj się z wynikiem `nsys`.

In [ ]:
!nvcc -o prefetch-to-cpu 01-vector-add.cu -run

Success! All values calculated correctly.


In [ ]:
!nsys profile --stats=true ./prefetch-to-cpu

Success! All values calculated correctly.

The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-bc2b-cfad-6b32-ecf0.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-bc2b-cfad-6b32-ecf0.qdrep"
Exporting 4673 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-bc2b-cfad-6b32-ecf0.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum            Name         
 -------  ---------------  ---------  ------------  ---------  -----------  ---------------------
    88.8      263,492,743          3  87,830,914.3     15,180  263,430,816  cudaMallocManaged    
     5.7       16,856,637          4   4,214,159.3    148,230   15,105,779  cudaMemPrefetchAsync 
     3.8       11,185,501   

Po użyciu _Memory Prefetching_ powinny być zauważalne zmiany, w postaci mniejszej ilości, ale większych transferów pamięci, a czas wykonywania kernela powinien być znacznie skrócony.

---
## Podsumowanie

Po zajęciach powinniście być w stanie:
- Używać narzędzia Nsight Systems (**nsys**) do profilowania aplikacji w celu zwiększania wydajności.
- Wykorzystywać wiedze na temat **Streaming Multiprocessors**, aby optymalizować konfiguracje wykonawcze.
- Rozumiec zachowanie **Unified Memory** w odniesieniu do migracji danych i `page faults`.
- Używać **Memory Prefetching** do zwiększania wydajności.
- Stosować iteracyjny proces projektowania, aby szybko przyspieszać i wdrażać aplikacje.

---
## Ćwiczenie końcowe: Optymalizacja aplikacji SAXPY

W pliku `01-saxpy.cu` znajduje się podstawowa wersja przyspieszonej aplikacji SAXPY. Obecnie zawiera kilka błędów, które należy znaleźć i naprawić, zanim będzie można pomyślnie skompilować, uruchomić, a następnie sprofilować ją za pomoca `nsys profile`.

Po naprawie błędów należy skupić się na profilowaniu aplikacji i odnotowywaniu czasu uruchamiania kernela `saxpy` w celu optymalizacji całej aplikacji. W tym celu należy wykorzystać zdobytą wiedzę. Zamiast wyszukiwać w poprzednich rozwiąniach zachęcam do wykorzystania [effortful retrieval](http://sites.gsu.edu/scholarlyteaching/effortful-retrieval/), co pozwoli na lepsze przyswojenie zdobytej wiedzy.

Celem jest modyfikacja rozwiązania tak, aby kernel `saxpy` wykonywał się w około *200us* bez modyfikacji `N` (w środowisku Google Colab). **Rozwiązanie należy przesłać na michal.zimon@wat.edu.pl.**

In [ ]:
!nvcc -o saxpy 01-saxpy.cu -run

c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 


In [ ]:
!nsys profile --stats=true ./saxpy

c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 

The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-067c-2b01-3a82-fcd1.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-067c-2b01-3a82-fcd1.qdrep"
Exporting 4481 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-067c-2b01-3a82-fcd1.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum            Name         
 -------  ---------------  ---------  ------------  ---------  -----------  ---------------------
    96.6      260,284,465          3  86,761,488.3     24,590  260,224,514  cudaMallocManaged    
     1.8        4,825,258          1